In [1]:
import pandas as pd
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import tensorflow as tf 
from sklearn.preprocessing import MinMaxScaler
from sklearn import preprocessing


2022-11-11 19:47:21.670091: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
df=pd.read_csv('/home/abhaylal/Desktop/Projects/4406b380-4ee6-4320-955b-eb9e6e2f0f44/data-new.csv')

In [3]:
df.drop(columns=['Unnamed: 0'],inplace=True)

In [4]:
df

,Unemployment,CPI,P/E,Open,High,Low,Close,Industrial Production,Treasury,Crash,year,month
0,6.4,6.837607,8.28,93.82,95.15,88.02,89.25,47.8693,7.956190,0,1978,1
1,6.3,6.429780,8.16,89.93,91.32,86.58,87.04,48.1283,8.033333,0,1978,2
2,6.3,6.554622,8.13,87.19,91.35,86.45,89.21,49.0553,8.035909,0,1978,3
3,6.1,6.500000,8.41,88.46,97.91,88.07,96.83,50.0260,8.154000,0,1978,4
4,6.0,6.965174,8.75,97.67,100.32,94.57,97.24,50.2717,8.345714,0,1978,5
...,...,...,...,...,...,...,...,...,...,...,...,...
523,5.2,5.251272,26.23,4406.86,4537.36,4367.73,4522.68,100.8370,1.283182,0,2021,8
524,4.7,5.390349,24.39,4528.80,4545.85,4305.91,4307.54,99.8498,1.374762,0,2021,9
525,4.6,6.221869,24.52,4317.16,4608.08,4278.94,4605.38,101.3602,1.582500,0,2021,10
526,4.2,6.809003,23.63,4610.62,4743.83,4560.00,4567.00,101.9610,1.559500,0,2021,11


In [5]:
df['Crash'].value_counts()

0    437
1     91
Name: Crash, dtype: int64

In [6]:
x=df.drop(columns="Crash")
y=df["Crash"]
x_train,x_test,y_train,y_test=train_test_split(x,y)

In [7]:
x_train=x_train.values
y_train=y_train.values

In [8]:
x_train = np.asarray(x_train)
x_test = np.asarray(x_test)

x_train_mean = np.mean(x_train)
x_train_std = np.std(x_train)

x_test_mean = np.mean(x_test)
x_test_std = np.std(x_test)

In [9]:
x_train = (x_train - x_train_mean)/x_train_std
x_test = (x_test - x_test_mean)/x_test_std

print(x_train.shape)

(396, 11)


In [10]:
x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], 1))
x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

In [11]:
x_train.shape


(396, 11, 1)

In [12]:
## CNN from keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Flatten, Dropout 
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard

In [13]:
cnn = Sequential()
cnn.add(Conv1D(512, 2, activation='relu', input_shape=(11,1)))
cnn.add(Dropout(0.2))

cnn.add(Conv1D(256, 2, activation='relu'))
cnn.add(Dropout(0.2))

cnn.add(Conv1D(128, 2, activation='relu'))
cnn.add(Dropout(0.2))

cnn.add(Conv1D(128, 2, activation='relu'))
cnn.add(Dropout(0.2))

cnn.add(Flatten())
cnn.add(Dropout(0.2))

cnn.add(Dense(1, activation='sigmoid'))

2022-11-11 19:47:43.257667: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-11-11 19:47:43.257699: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (pop-os): /proc/driver/nvidia/version does not exist
2022-11-11 19:47:43.258130: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [14]:
cnn.summary()
cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics = ['accuracy'])
history = cnn.fit(x_train , y_train , verbose=1 , batch_size=32 , epochs=100 ,validation_data=(x_test, y_test) )

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 10, 512)           1536      
                                                                 
 dropout (Dropout)           (None, 10, 512)           0         
                                                                 
 conv1d_1 (Conv1D)           (None, 9, 256)            262400    
                                                                 
 dropout_1 (Dropout)         (None, 9, 256)            0         
                                                                 
 conv1d_2 (Conv1D)           (None, 8, 128)            65664     
                                                                 
 dropout_2 (Dropout)         (None, 8, 128)            0         
                                                                 
 conv1d_3 (Conv1D)           (None, 7, 128)            3

In [15]:
loss, accuracy = cnn.evaluate(x_test, y_test, verbose=1)
loss_v, accuracy_v = cnn.evaluate(x_train, y_train, verbose=1)
print("Train: accuracy = %f  ;  loss_v = %f" % (accuracy_v, loss_v))
print("Test: accuracy = %f  ;  loss = %f" % (accuracy, loss))


13/13 [==============================] - 0s 2ms/step - loss: 0.3744 - accuracy: 0.8566
Train: accuracy = 0.856571  ;  loss_v = 0.374418
Test: accuracy = 0.871212  ;  loss = 0.337944


In [16]:
cnn.save('cnn.h5')